In [6]:
from tensorflow import keras
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
import pickle
import skopt

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
N_CLASSES = 10

In [4]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [15]:
# Preprocess data for cnn
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [21]:
def build_model(n_units=[50], n_layers=1, activation='relu', optimizer='adam', learning_rate=1e-3, ae_layers=1):
    layer_map = { 1: 'one', 2: 'two', 3: 'three', 4: 'four' }
    ae_loaded = keras.models.load_model(f'./models/ae_{layer_map[ae_layers]}_layers/')

    # Freeze all AE layer weights
    for l in ae_loaded.layers:
        l.trainable = False

    # Transfer only the encoder layers
    model = keras.models.Sequential(ae_loaded.layers[:-ae_layers])

    # Add new layers
    for i in range(n_layers):
        model.add(keras.layers.Dense(
            units=n_units[i],
            activation=activation,
            name=f'hidden_layer_{i}'
        ))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax', name='output_layer'))

    opt = None
    if optimizer == 'adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        opt = keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(
        optimizer=opt,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
for i in range(10000):
    print('\r')